## Initialize environment

In [ ]:
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env")

## Initialize Embeddings & Vector Store

We use AWS Bedrock Embeddings and Azure AI search as vector store.

In [ ]:
import os
from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.vectorstores.azuresearch import AzureSearch


class AWSBedrockEmbeddings:

    def __init__(self):
        self._embeddings = None
        self._validate_aws_env_variables()
        self._region_name = os.environ["AWS_REGION"]
        self._model_id = os.environ["AWS_LLM_EMBEDDINGS_ID"]
        self.initialize_embeddings()

    def initialize_embeddings(self):
        self._embeddings = BedrockEmbeddings(region_name=self._region_name, model_id=self._model_id)

    @property
    def region_name(self):
        return self._region_name

    @property
    def model_id(self):
        return self._model_id

    @property
    def embeddings(self):
        return self._embeddings

    def _validate_aws_env_variables(self):
        if "AWS_REGION" not in os.environ:
            raise ValueError("AWS_REGION environment variable not set")
        if "AWS_LLM_EMBEDDINGS_ID" not in os.environ:
            raise ValueError("AWS_LLM_EMBEDDINGS_ID environment variable not set")
        if "AWS_ACCESS_KEY_ID" not in os.environ:
            raise ValueError("AWS_ACCESS_KEY_ID environment variable not set")
        if "AWS_SECRET_ACCESS_KEY" not in os.environ:
            raise ValueError("AWS_SECRET_ACCESS_KEY environment variable not set")


def get_azure_search_vector_store(embeddings):
    """
    Get the Azure Search vector store using the provided embeddings.

    :param embeddings: The embeddings to be used for the vector store.
    :return: The Azure Search vector store.
    """
    azure_search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
    azure_search_api_key = os.getenv("AZURE_SEARCH_API_KEY")
    azure_search_index = os.getenv("AZURE_SEARCH_INDEX")

    vector_store = AzureSearch(
        azure_search_endpoint=azure_search_endpoint,
        azure_search_key=azure_search_api_key,
        index_name=azure_search_index,
        embedding_function=embeddings.embed_query,
    )
    return vector_store

In [ ]:
embeddings = AWSBedrockEmbeddings().embeddings
vectorstore = get_azure_search_vector_store(embeddings)

## Setup Caching via SQLAlchemy

**References:**
https://python.langchain.com/docs/integrations/llms/llm_caching#sqlalchemy-cache

### Custom SQLAlchemy Schemas

In [ ]:
# You can define your own declarative SQLAlchemyCache child class to customize the schema used for caching. For example, to support high-speed fulltext prompt indexing with Postgres, use:

from langchain.cache import SQLAlchemyCache
from sqlalchemy import Column, Computed, Index, Integer, Sequence, String, create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy_utils import TSVectorType

Base = declarative_base()


class FulltextLLMCache(Base):  # type: ignore
    """Postgres table for fulltext-indexed LLM Cache"""

    __tablename__ = "llm_cache_fulltext"
    id = Column(Integer, Sequence("cache_id"), primary_key=True)
    prompt = Column(String, nullable=False)
    llm = Column(String, nullable=False)
    idx = Column(Integer)
    response = Column(String)
    # https://www.postgresql.org/docs/current/datatype-textsearch.html
    # https://dev.to/nightbird07/full-text-search-in-postgresql-a-comprehensive-guide-3kcn
    # https://medium.com/geekculture/comprehend-tsvector-and-tsquery-in-postgres-for-full-text-search-1fd4323409fc
    prompt_tsv = Column(
        TSVectorType(),
        Computed("to_tsvector('english', llm || ' ' || prompt)", persisted=True),
    )
    __table_args__ = (
        Index("idx_fulltext_prompt_tsv", prompt_tsv, postgresql_using="gin"),
    )


postgres_host = os.getenv("POSTGRES_HOST")
postgres_port = os.getenv("POSTGRES_PORT")
postgres_user = os.getenv("POSTGRES_USER")
postgres_password = os.getenv("POSTGRES_PASSWORD")
postgres_db = os.getenv("POSTGRES_DB")

sql_url = f"postgresql://{postgres_user}:{postgres_password}@{postgres_host}:{postgres_port}/{postgres_db}"

engine = create_engine(sql_url)


### Configure caching via Langchain

In [ ]:
from langchain_core.globals import set_llm_cache

set_llm_cache(SQLAlchemyCache(engine, FulltextLLMCache))

In [ ]:
from langchain_openai import AzureOpenAI

azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_LLM_MODEL_DEPLOYMENT_NAME")
openai_api_key = os.getenv("AZURE_API_KEY")
openai_api_version= os.getenv("AZURE_API_VERSION")

llm = AzureOpenAI(azure_endpoint=azure_endpoint, 
                  deployment_name=deployment_name,
                  openai_api_key=openai_api_key,
                  openai_api_version=openai_api_version)

In [ ]:
%%time

llm.invoke("Tell me a joke")